In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5110)
/usr/local/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [3]:
!pip install bcolz


    100% |████████████████████████████████| 1.3MB 1.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/e9/84/eb/f8f3caa627bb01ebc96034c3411f59870951246e5873b3f4c7
Successfully built bcolz


In [4]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [5]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

## Setup

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [6]:
#path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open("duda.txt").read().lower()
print('corpus length:', len(text))

corpus length: 139932


In [7]:
!tail {"duda.txt"} -n25

In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 78


In [9]:
chars.insert(0, "\0")

In [10]:
''.join(chars[1:-6])

'\n\r !"%()+,-.0123456789:?abcdefghijklmnopqrstuvwxyz\x80\x81\x82\x84\x85\x86\x87\x92\x93\x99\x9a\x9b\x9d\x9e\xa1\xad\xb3\xba\xbb\xbc\xbf'

In [11]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [12]:
idx = [char_indices[c] for c in text]

In [13]:
idx[:10]

[77, 69, 71, 43, 50, 25, 38, 39, 47, 38]

In [14]:
''.join(indices_char[i] for i in idx[:70])

'\xef\xbb\xbfszanowni pa\xc5\x84stwo marsza\xc5\x82kowie i marsza\xc5\x82kowie seniorzy,\r\nszanowna'

## Preprocess and create model

In [22]:
maxlen = 43
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 139890


In [24]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [25]:
sentences.shape, next_chars.shape

((139888, 43), (139888, 43))

In [26]:
n_fac = 24

In [27]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        BatchNormalization(),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

In [28]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam() )

## Train

In [29]:
def print_example():
    seed_string="szanowny panie marszałku i szanowni zebrani"
    for i in range(500):
        x=np.array([char_indices[c] for c in seed_string[-8:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [30]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=32, nb_epoch=1, verbose = 2)

Epoch 1/1
461s - loss: 1.3283


In [20]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=32, nb_epoch=1, verbose =2)

Epoch 1/1
2019s - loss: 0.9727


In [33]:
model.optimizer.lr=0.001

In [34]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose = 2)

Epoch 1/1
275s - loss: 0.9625


In [36]:
model.optimizer.lr=0.0001

In [37]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose = 2)

Epoch 1/1
275s - loss: 0.8766


In [38]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=5, verbose = 2)

Epoch 1/5
275s - loss: 0.8220
Epoch 2/5
275s - loss: 0.7851
Epoch 3/5
274s - loss: 0.7577
Epoch 4/5
274s - loss: 0.7356
Epoch 5/5
274s - loss: 0.7175


In [40]:
model.save_weights('char_dudaGPU.h5')